In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd
import numpy as np

In [5]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/빅데프/BDP_Project/gas_data'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [7]:
main = pd.read_csv("gas_data/main.csv", encoding="cp949")

# Main 데이터 전처리

In [8]:
main['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [9]:
#main의 구분
d_map={}
for i, d in enumerate(main['구분'].unique()):
  d_map[d] = i

#A,B,C,D,E,F,..를 0,1,2,3,4,5,6으로 구분
main['구분'] = main['구분'].map(d_map)
main['구분']

0         0
1         0
2         0
3         0
4         0
         ..
368083    6
368084    6
368085    6
368086    6
368087    6
Name: 구분, Length: 368088, dtype: int64

In [10]:
main['연월일'] = pd.to_datetime(main['연월일'])
main['year'] = main['연월일'].dt.year
main['month'] = main['연월일'].dt.month
main['day'] = main['연월일'].dt.day

In [11]:
def outliers_processing(df):
  q1, q3 = np.percentile(df, [25, 75])
  iqr = q3-q1
  lower_bound = q1-(iqr*1.5)
  upper_bound = q3 + (iqr*1.5)
  return lower_bound, upper_bound

In [13]:
main_by_sections = {}
for i in range(7):
  main_by_sections[i] = main[main['구분'] == i]
main_by_sections

{0:               연월일  시간  구분       공급량  year  month  day
 0      2013-01-01   1   0  2497.129  2013      1    1
 1      2013-01-01   2   0  2363.265  2013      1    1
 2      2013-01-01   3   0  2258.505  2013      1    1
 3      2013-01-01   4   0  2243.969  2013      1    1
 4      2013-01-01   5   0  2344.105  2013      1    1
 ...           ...  ..  ..       ...   ...    ...  ...
 315523 2018-12-31  20   0  3040.868  2018     12   31
 315524 2018-12-31  21   0  2973.577  2018     12   31
 315525 2018-12-31  22   0  2840.272  2018     12   31
 315526 2018-12-31  23   0  2692.385  2018     12   31
 315527 2018-12-31  24   0  2583.340  2018     12   31
 
 [52584 rows x 7 columns],
 1:               연월일  시간  구분       공급량  year  month  day
 8760   2013-01-01   1   1  2169.093  2013      1    1
 8761   2013-01-01   2   1  2051.733  2013      1    1
 8762   2013-01-01   3   1  1923.253  2013      1    1
 8763   2013-01-01   4   1  1927.253  2013      1    1
 8764   2013-01-01   5   1  20

In [14]:
lower = {}
upper = {}
for i in range(7):
  lower[i] = {}
  upper[i] = {}
  for j in range(12):
    lower[i][j], upper[i][j]= outliers_processing(main_by_sections[i][main_by_sections[i]['month'] == j+1]['공급량'])



In [15]:
main['lower'] =""
main['upper'] =""

In [16]:
for i in range(7):
  for j in range(12):
    main.loc[(main['구분']==i)&(main['month']==j+1),'lower' ] = lower[i][j]
    main.loc[(main['구분']==i)&(main['month']==j+1),'upper' ] = upper[i][j]

In [17]:
main['이상치처리_공급량'] =""

In [18]:
main['이상치처리_공급량'] = main.apply(lambda x: x['lower'] if x['공급량'] < x['lower'] else x['upper'] if x['공급량'] > x['upper'] else x['공급량'], axis = 1)

In [19]:
main['공급량'] = main['이상치처리_공급량']
main = main.drop(['lower', 'upper', '이상치처리_공급량'], axis = 1)
main

,연월일,시간,구분,공급량,year,month,day
0,2013-01-01,1,0,2497.129,2013,1,1
1,2013-01-01,2,0,2363.265,2013,1,1
2,2013-01-01,3,0,2258.505,2013,1,1
3,2013-01-01,4,0,2243.969,2013,1,1
4,2013-01-01,5,0,2344.105,2013,1,1
...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31
368084,2018-12-31,21,6,669.961,2018,12,31
368085,2018-12-31,22,6,657.941,2018,12,31
368086,2018-12-31,23,6,610.953,2018,12,31


In [20]:
from sklearn.model_selection import train_test_split

target = main['공급량']
main = main.drop(['공급량'], axis = 1)
data = main

x_train, x_text, y_train, y_test = train_test_split(data, target, test_size=0.3, shuffle=True, random_state=34)